In [1]:
import openclsim.model as model
import saint_brieuc
import simpy
import shapely
import datetime
import pandas as pd

In [15]:
class InstallationSimulation(object): 
    def __init__(self, start_date: datetime.datetime, size: int = 1):
        
        # Initialise simpy simulation environment
        POSIX = start_date.timestamp()
        self.env = simpy.Environment(initial_time=POSIX)
        
        # Simulation attributes
        self.size = size

    def create_entities(self):
        
        # Define source location
        port_of_cherbourg = saint_brieuc.Site(
            env=self.env,  # SimPy environment.
            name='PORT OF CHERBOURG',  # Source reference name.
            ID='PORT',  # Object identification.
            geometry=shapely.geometry.Point(0, 0),   # Global position.
            capacity=self.size,  # Max. number of objects in store.
            level=self.size  # Current number of objects hold in store.        
        )

        # Define construction site
        offshore_wind_farm = saint_brieuc.Site(
            env=self.env,  # SimPy environment.
            name='OWF SAINT BRIEUC',  # Source reference name.
            ID='OWF',  # Object identification.
            geometry=shapely.geometry.Point(0, 0),   # Global position.
            capacity=self.size,  # Max. number of objects in store.
            level=0  # Current number of objects hold in store.        
        )

        # Define installation equipment
        aeolus = saint_brieuc.InstallationVessel(
            env=self.env,  # SimPy environment.
            name='AEOLUS',  # Equipment reference name.
            ID='INSTALLATION_VESSEL',  # Object identification.
            geometry=shapely.geometry.Point(0, 0),   # Global position.
            capacity=5,  # Max. number of objects onboard.
            level=0  # Current number of objects hold onboard.
        )

        return port_of_cherbourg, offshore_wind_farm, aeolus

    def define_activities(self):

        # Create entities
        port, owf, mover = self.create_entities()
        
        # Define global activities
        transit = saint_brieuc.MoveActivity(
            env=self.env,  # SimPy environment.
            name='TRANSIT BETWEEN SOURCE AND OWF',  # Activity description.
            ID='TRANSIT',  # Activity identification.
            destination=owf,  # Where equipment and objects are moved to.
            mover=mover,  # The equipment used to move objects.
            duration=3600,  # Time it takes to transit.
            postpone_start=True  # Defines activity to be a sub-process. 
        )

        relocate = saint_brieuc.MoveActivity(
            env=self.env,  # SimPy environment.
            name='TRANSIT BETWEEN SITES',  # Activity description.
            ID='RELOCATE',  # Activity identification.
            destination=owf,  # Where equipment and objects are moved to.
            mover=mover,  # The equipment used to move objects.
            duration=1800,  # Time it takes to transit.
            postpone_start=True  # Defines activity to be a sub-process. 
        )

        load = saint_brieuc.TransferActivity(
            env=self.env,  # SimPy environment.
            name='LOAD COMPONENT FROM SOURCE TO DECK',  # Activity description.
            ID='TRANSFER',  # Activity identification.  
            origin=port,  # Where objects are moved from.
            destination=mover,  # Where objects are moved to.
            processor=mover,  # Who's shifting the objects.
            duration=3600,  # Time it takes to perform activity.
            postpone_start=True  # Defines activity to be a sub-process.
        )

        install = saint_brieuc.TransferActivity(
            env=self.env,  # SimPy environment.
            name='INSTALL COMPONENT FROM DECK TO OWF',  # Activity description.
            ID='TRANSFER',  # Activity identification.
            origin=mover,  # Where objects are moved from.
            destination=owf,  # Where objects are moved to.
            processor=mover,  # Who's shifting the objects.
            duration=3600,  # Time it takes to perform activity.
            postpone_start=True  # Defines activity to be a sub-process
        )

        # Define cyclic activities
        load_expr = [{'or': [
                {
                    'type': 'container',
                    'concept': mover,
                    'state': 'full'
                },
                {
                    'type': 'container',
                    'concept': port,
                    'state': 'empty'
                }
            ]
        }]

        load_cycle = model.WhileActivity(
            env=self.env,
            name='LOAD WHILE CYCLE',
            ID='TRANSFER',
            registry={},
            sub_processes=[load],
            condition_event=load_expr,
            postpone_start=True
        )

        unload_expr = [{
            'or': [
                {
                    'type': 'container',
                    'concept': owf,
                    'state': 'full'
                },
                {
                    'type': 'container',
                    'concept': mover,
                    'state': 'empty'
                }
            ]
        }]

        unload = model.SequentialActivity(
            env=self.env,
            name='INSTALLATION SEQUENTIAL CYCLE',
            ID='TRANSFER',
            registry={},
            sub_processes=[install, relocate],
            postpone_start=True
        )

        unload_cycle = model.WhileActivity(
            env=self.env,
            name='INSTALLATION WHILE CYCLE',
            ID='TRANSFER',
            registry={},
            sub_processes=[unload],
            condition_event=unload_expr,
            postpone_start=True
        )

        # Main operation process
        operation_cycle = [load_cycle, transit, unload_cycle, transit]
        main_expression = [{'type': 'container', 'concept': owf, 'state': 'full'}]

        operation_cycle = model.SequentialActivity(
            env=self.env,
            name='MAIN SEQUENTIAL PROCESS',
            ID='MAIN',
            registry={},
            sub_processes=operation_cycle,
            postpone_start=True
        )

        activities = model.WhileActivity(
            env=self.env,
            name='MAIN WHILE CYCLE',
            ID='MAIN',
            registry={},
            sub_processes=[operation_cycle],
            condition_event=main_expression,
            postpone_start=False
        )

        # List the individual activities for logging purposes
        activity_list = [transit, relocate, load, install]

        return activity_list

    
    def start_simulation(self):
        
        # Define installation activities
        self.activity_list = self.define_activities()
        
        # Execute simulation
        self.env.run()

    @property
    def event_log(self):
        return saint_brieuc.get_event_log(self.activity_list)

In [16]:
sim = InstallationSimulation(start_date=datetime.datetime(2021, 1, 1), size=6)
sim.start_simulation()
sim.event_log

,Timestamp,ActivityID,ActivityState
0,2020-12-31 23:00:00,TRANSFER,START
1,2021-01-01 00:00:00,TRANSFER,STOP
2,2021-01-01 00:00:00,TRANSFER,START
3,2021-01-01 01:00:00,TRANSFER,STOP
4,2021-01-01 01:00:00,TRANSFER,START
5,2021-01-01 02:00:00,TRANSFER,STOP
6,2021-01-01 02:00:00,TRANSFER,START
7,2021-01-01 03:00:00,TRANSFER,STOP
8,2021-01-01 03:00:00,TRANSFER,START
9,2021-01-01 04:00:00,TRANSFER,STOP
